This is a proof of concept for a password bank system that securely stores and retrieves user passwords using encryption. We will cryptography to show how the mathematical principles behind encryption work in practice.

# Encrypt ceasar cipher

In [1]:
# Spør brukeren om tekst og shift-verdi
text = input("Skriv inn teksten du vil kryptere: ")
shift = int(input("Oppgi shift-verdi (f.eks. 3): "))

# Enkel Caesar cipher-funksjon
def caesar_cipher(text, shift):
    result = ""
    for char in text:
        if char.isalpha():
            base = 'A' if char.isupper() else 'a'
            result += chr((ord(char) - ord(base) + shift) % 26 + ord(base))
        else:
            result += char
    return result

encrypted = caesar_cipher(text, shift)
print(f"Kryptert tekst: {encrypted}")


Kryptert tekst: khl



# Decrypt ceasar cipher


In [2]:
def caesar_decrypt(ciphertext, shift):
    decrypted = ""
    for char in ciphertext:
        if char.isalpha():
            base = 'A' if char.isupper() else 'a'
            decrypted += chr((ord(char) - ord(base) - shift) % 26 + ord(base))
        else:
            decrypted += char
    return decrypted


# Bruker output fra krypteringsdelen over
decrypted_text = caesar_decrypt(encrypted, shift)
print(f"Dekryptert tekst: {decrypted_text}")


Dekryptert tekst: hei



# Affine cipher
Source for this code is https://www.geeksforgeeks.org/dsa/implementation-affine-cipher/ and should be modified

In [5]:
# Find coprime numbers for 26

import math

def find_coprime_numbers(m):
    coprimes = [a for a in range(1, m) if math.gcd(a, m) == 1]
    return coprimes

print("Numbers coprime with 26:")
print(find_coprime_numbers(26))


Numbers coprime with 26:
[1, 3, 5, 7, 9, 11, 15, 17, 19, 21, 23, 25]


In [7]:
# Affine Cipher Implementation with User Input
# Based on GeeksforGeeks example, modified for interactivity and clarity

# Extended Euclidean Algorithm to find modular inverse
def egcd(a, b):
    x, y, u, v = 0, 1, 1, 0
    while a != 0:
        q, r = b // a, b % a
        m, n = x - u * q, y - v * q
        b, a, x, y, u, v = a, r, u, v, m, n
    gcd = b
    return gcd, x, y

def modinv(a, m):
    gcd, x, y = egcd(a, m)
    if gcd != 1:
        return None  # Modular inverse does not exist
    else:
        return x % m

# Encryption function
def affine_encrypt(text, key):
    a, b = key
    return ''.join([
        chr(((a * (ord(t) - ord('A')) + b) % 26) + ord('A'))
        for t in text.upper().replace(' ', '')
    ])

# Decryption function
def affine_decrypt(cipher, key):
    a, b = key
    a_inv = modinv(a, 26)
    if a_inv is None:
        raise ValueError("Invalid key: 'a' must be coprime with 26.")
    return ''.join([
        chr(((a_inv * ((ord(c) - ord('A')) - b)) % 26) + ord('A'))
        for c in cipher
    ])

# Main program
def main():
    print("=== Affine Cipher Encryption/Decryption ===")
    text = input("Enter text: ").strip()
    a = int(input("Enter key 'a' (must be coprime with 26): "))
    b = int(input("Enter key 'b': "))

    # Check that 'a' is valid
    if egcd(a, 26)[0] != 1:
        print("Invalid key 'a'. It must be coprime with 26.")
        return

    encrypted = affine_encrypt(text, (a, b))
    decrypted = affine_decrypt(encrypted, (a, b))

    print(f"\nEncrypted text: {encrypted}")
    print(f"Decrypted text: {decrypted}")

if __name__ == "__main__":
    main()


=== Affine Cipher Encryption/Decryption ===

Encrypted text: WKBCPGAN
Decrypted text: SUPERMAN


# Encrypt RSA cipher

In [1]:
# RSA proof-of-concept for the report / appendix.
# Educational code, not suitable for production use
# (no padding, no side-channel protections, etc.).

import secrets
from math import gcd

In [4]:
def egcd(a: int, b: int):
    """
    Extended Euclidean Algorithm.
    Returns (g, x, y) such that ax + by = g = gcd(a, b).
    """
    if b == 0:
        return a, 1, 0
    else:
        g, x1, y1 = egcd(b, a % b)
        # Back-substitute
        x = y1
        y = x1 - (a // b) * y1
        return g, x, y


def modinv(a: int, m: int) -> int:
    """
    Modular inverse of a modulo m.
    Returns x such that a*x ≡ 1 (mod m), if gcd(a, m) = 1.
    Raises ValueError if inverse does not exist.
    """
    g, x, _ = egcd(a, m)
    if g != 1:
        raise ValueError(f"No modular inverse for a={a} modulo m={m}")
    return x % m


In [5]:
def is_probable_prime(n: int, k: int = 10) -> bool:
    """
    Miller–Rabin probabilistic primality test.
    Returns True if n is probably prime, False if composite.
    k is the number of rounds.
    """
    if n < 2:
        return False
    # Handle small primes explicitly
    small_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23]
    if n in small_primes:
        return True
    if any(n % p == 0 for p in small_primes):
        return False

    # Write n-1 as 2^r * d with d odd
    r = 0
    d = n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    for _ in range(k):
        a = secrets.randbelow(n - 3) + 2  # random in [2, n-2]
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True


def generate_prime(bits: int) -> int:
    """
    Generate a random prime with the given bit length.
    Uses Miller–Rabin to test primality.
    """
    if bits < 2:
        raise ValueError("Bit length must be at least 2")

    while True:
        # Generate random odd candidate
        candidate = secrets.randbits(bits) | 1
        # Ensure top bit is set so we truly get 'bits'-bit numbers
        candidate |= (1 << (bits - 1))
        if is_probable_prime(candidate):
            return candidate


In [6]:
def generate_rsa_keypair(bits: int = 512):
    """
    Generate an RSA key pair of approximately 'bits' bits.
    - p and q are primes of ~bits/2 bits.
    - n = p*q
    - phi = (p-1)(q-1)
    - e chosen coprime with phi (default 65537 if possible)
    - d = e^{-1} mod phi
    """
    # Choose two distinct primes
    half_bits = bits // 2
    p = generate_prime(half_bits)
    q = generate_prime(half_bits)
    while q == p:
        q = generate_prime(half_bits)

    n = p * q
    phi = (p - 1) * (q - 1)

    # Standard public exponent
    e = 65537
    if gcd(e, phi) != 1:
        # Fallback: find another e
        e = 3
        while gcd(e, phi) != 1:
            e += 2

    d = modinv(e, phi)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key, (p, q, phi)


# Example: generate a 512-bit key pair
public_key, private_key, internal_values = generate_rsa_keypair(bits=512)
n, e = public_key
_, d = private_key

print("Public key (n, e):")
print("n =", n)
print("e =", e)
print()
print("Private exponent d:")
print("d =", d)


Public key (n, e):
n = 8024487662005005489309760650877911432431320642717317785199925625570323608543881464051784825326176895371523268118469389412673913778185650808845037174147391
e = 65537

Private exponent d:
d = 4450280430340630933891581253594283670646333827917109910750820098371591343456816246138312575114922863392118176855570370424434979897390612228098417573225809


In [7]:
def encrypt_int(m: int, public_key: tuple[int, int]) -> int:
    """
    Encrypt an integer m with public key (n, e).
    Requires 0 ≤ m < n.
    """
    n, e = public_key
    if not (0 <= m < n):
        raise ValueError("Message integer out of range (must be < n)")
    return pow(m, e, n)


def decrypt_int(c: int, private_key: tuple[int, int]) -> int:
    """
    Decrypt an integer c with private key (n, d).
    """
    n, d = private_key
    return pow(c, d, n)


def text_to_int(text: str) -> int:
    """
    Encode a UTF-8 string as a big-endian integer.
    """
    data = text.encode("utf-8")
    return int.from_bytes(data, byteorder="big")


def int_to_text(m: int) -> str:
    """
    Decode a big-endian integer back to a UTF-8 string.
    """
    if m == 0:
        return ""
    # Compute the minimum number of bytes needed
    length = (m.bit_length() + 7) // 8
    data = m.to_bytes(length, byteorder="big")
    return data.decode("utf-8")


In [8]:
message = "Hello RSA from the password bank"

print("Original message:")
print(message)

# Encode message as integer
m = text_to_int(message)
print("\nMessage as integer m:")
print(m)

# Encrypt
c = encrypt_int(m, public_key)
print("\nCiphertext integer c:")
print(c)

# Decrypt
m_dec = decrypt_int(c, private_key)
print("\nDecrypted integer m_dec:")
print(m_dec)

# Convert back to text
decoded_message = int_to_text(m_dec)
print("\nDecoded message:")
print(decoded_message)

print("\nDecryption correct?", decoded_message == message)


Original message:
Hello RSA from the password bank

Message as integer m:
32745724963520450916509782688602831250971793034266645229058059736080653053547

Ciphertext integer c:
1342550058278070684528233414315927681231596138008398332794147794920330754496065446433862169205475675202788174076930336809900727606742747857454412511305649

Decrypted integer m_dec:
32745724963520450916509782688602831250971793034266645229058059736080653053547

Decoded message:
Hello RSA from the password bank

Decryption correct? True


In [9]:
# Toy RSA example with small primes (not secure, just for illustration)

p = 61
q = 53
n = p * q
phi = (p - 1) * (q - 1)

e = 17  # must satisfy gcd(e, phi) = 1
print("gcd(e, phi) =", gcd(e, phi))

d = modinv(e, phi)

print("Toy RSA parameters:")
print("p =", p)
print("q =", q)
print("n =", n)
print("phi =", phi)
print("e =", e)
print("d =", d)

P = 65  # example plaintext integer (0 ≤ P < n)
C = pow(P, e, n)
P_dec = pow(C, d, n)

print("\nPlaintext P:", P)
print("Ciphertext C:", C)
print("Decrypted P_dec:", P_dec)
print("Decryption correct?", P == P_dec)


gcd(e, phi) = 1
Toy RSA parameters:
p = 61
q = 53
n = 3233
phi = 3120
e = 17
d = 2753

Plaintext P: 65
Ciphertext C: 2790
Decrypted P_dec: 65
Decryption correct? True
